In [1]:
# Calculate a cumulative slope on arc network
import arcpy
import pandas as pd

In [3]:
###############
# Load a feature class and extract table data into pandas

In [2]:
# Get the TransRefEdges info
fc = r"D:\slope\slope.gdb\slope"
bike = r"D:\slope\slope.gdb\bike_fac"
out_fc = r"D:\slope\slope.gdb\slope_segment_new"
# Get the bike facility data

In [3]:
cursor = arcpy.da.UpdateCursor (fc, ["PSRCEdgeID"])   

In [ ]:
# for row in cursor:
#     print row[0]

In [50]:
bike

'D:\\slope\\slope.gdb\\bike_fac'

In [53]:
# Send feature class to numpy array
# Get the TransRefEdges data
my_array = arcpy.da.FeatureClassToNumPyArray(fc, ('PSRCEdgeId'))

# Get bike attributes (exported as a FC from the related table)
bike_array = arcpy.da.FeatureClassToNumPyArray(bike, field_names=['PSRCEdgeId', 'IJBikeFacil', 'JIBikeFacil'])

# We can also export slope once it's attached to the gdb

In [55]:
# Load the np array into pandas
base = pd.DataFrame(my_array)
bike = pd.DataFrame(bike_array)

In [56]:
# Join bike attributes to the base df
df = base.merge(bike)

In [60]:
###############
# Split a polyline into equal sized segments

In [134]:
# Use a small version of the transRefEdges for testing
# in_fc = r"D:\slope\test.gdb\test_transRefEdges"
in_fc = r"D:\slope\test.gdb\transRefEdges"

In [190]:
fields = arcpy.ListFields(in_fc, "", "")

In [191]:
# for i in xrange(len(fields)):
#     print fields[i].name

# try storing points in a dictionary
results = {}

In [192]:
# Add a new field to the 
# arcpy.AddField_management(fc, "Name", "TEXT", field_length = 50)

In [188]:
# fields = arcpy.ListFields(in_fc, "", "")
# for field in fields: print field.name
# # fields

In [135]:
# Loop through each polyline and split the line into points, saving them in points array
points = []
sr = arcpy.Describe(in_fc).spatialReference
counter = 0

final_result = []

with arcpy.da.SearchCursor(in_fc,["SHAPE@",'PSRCEdgeID'], spatial_reference=sr) as cursor:  
    for row in cursor:
        
        # Set the step based on the line's length
        #print row[0].length
        # How many times must the line be split
        # Assuming we want 10 foot segments
        segment_len = 10
        split_count = int(row[0].length/segment_len)
        #print 'step: ' + str(step)
        #print "edgeID: " + str(row[1])
        # list of points for each edgeId
        big_output = []
        for i in range(split_count):
            point = row[0].positionAlongLine(i*segment_len)
            points.append(point)
            x = point.firstPoint.X
            y = point.firstPoint.Y
            point_list = []
            point_list.append(x)
            point_list.append(y)
            output = (str(row[1]), (x, y))
            final_result.append(output)
            #local_points.append(points)
        #my_dict[row[1]] = local_points

KeyboardInterrupt: 

In [96]:
# Send final result to a feature class

# First make a new empty feature class
arcpy.CreateFeatureclass_management(out_path=r'D:\slope\test.gdb', out_name='tupletest1', 
                                    geometry_type='POINT', template='',
                                    spatial_reference=sr)

<Result 'D:\\slope\\test.gdb\\tupletest1'>

In [102]:
arcpy.env.workspace = r'D:\slope\test.gdb'

In [107]:
# Add a column so we can add the EdgeID
arcpy.AddField_management('tupletest1', "PSRCEdgeID", "LONG", field_length = 50)

<Result 'D:\\slope\\test.gdb\\tupletest1'>

In [113]:
cursor = arcpy.da.InsertCursor(r'D:\slope\test.gdb\tupletest1', ('PSRCEdgeID', 'SHAPE@XY'))

In [116]:
# Add a row for each point that we created
# for row in final_result:
#     cursor.insertRow(row)
# del cursor

NameError: name 'cursor' is not defined

In [68]:
other = []
other.append(test)
other

[('1', (2, 3))]

In [57]:
# point_list = 
# point_list.append('xvalue')
# point_list.append('yvalue')
# a_list = tuple(point_list)
my_thing = ('xvalue', 'yvalue')

In [61]:
test = tuple(my_thing)

In [62]:
test

('xvalue', 'yvalue')

In [201]:
# Export these points as a feature class
point_out = r'\fresh_points'
arcpy.CopyFeatures_management(points, r"D:\slope\test.gdb" + point_out) 

<Result 'D:\\slope\\test.gdb\\fresh_points'>

In [ ]:
# Note: cannot just intersect points and lines because the order of the 
# point creation is lost! Probably because lat long algorithm behind the intersect
# Need to tag line informatin onto the points data instead...

In [204]:
# Before calculating slopes, intersect the points and the TransRefEdges file to get PSRCEdgeID for each point
########### NO LONGER USING< DOESN't WORK!
# arcpy.env.workspace = r'D:\slope\test.gdb'
# arcpy.Intersect_analysis(in_features=['fresh_points', 'test_transRefEdges'], 
#                          out_feature_class= r'D:\slope\test.gdb\points_edges_join', 
#                          output_type='POINT')

<Result 'D:\\slope\\test.gdb\\points_edges_join'>

In [ ]:
# Extract raster elevation value to the points to get an elevation field
# Must be done manually because spatial analyst license problems
# arcpy.ExtractValuesToPoints(in_point_features='points_edges_join', 
#                             in_raster='PSRC30ft_edited_h1', 
#                             out_point_features='points_with_elevation')


In [214]:
# The current raster is at 30ft intervals (i.e., 30x30' squares of elevation variation). Elevation should be in feet for this raster.
# May be updating this data at some point.
# Saving output as "points_with_elevation" in the test gdb

In [215]:
# The result places elevation in the "RASTERVALU" column.

In [216]:
# Now we can process the results in pandas/numpy

In [2]:
# Load in the shapefile of points with elevation
elevation_shp = arcpy.da.FeatureClassToNumPyArray(r"D:\slope\test.gdb\full_net_with_elevation", ('RASTERVALU','PSRCEdgeID'))

In [21]:
# Load results as a dataframe and group results by Edge ID
df = pd.DataFrame(elevation_shp)

In [5]:
len(df)

15489769

In [23]:
# Ach! We also need edge length
# Load in the edges shapefile with numpy and join in pandas?
edges = arcpy.da.FeatureClassToNumPyArray(r"D:\slope\test.gdb\test_transRefEdges", ('PSRCEdgeID', 'Shape_Length'))
edges_df = pd.DataFrame(edges)

# Now merge the two dfs
df = df.merge(edges_df)

In [24]:
# Get the length
df['edgeID']= df['PSRCEdgeID']
length = df.groupby('PSRCEdgeID').min()[['Shape_Length','edgeID']]

In [25]:
# Get a list of all unique Edge IDs
edges = df.groupby('PSRCEdgeID').mean().index

In [26]:
df

,RASTERVALU,PSRCEdgeID,Shape_Length,edgeID
0,21.292835,39,940.034381,39
1,21.292835,39,940.034381,39
2,21.292835,39,940.034381,39
3,21.632460,39,940.034381,39
4,21.632460,39,940.034381,39
5,21.632460,39,940.034381,39
6,21.632460,39,940.034381,39
7,22.684933,39,940.034381,39
8,22.684933,39,940.034381,39
9,22.684933,39,940.034381,39


In [ ]:
# NOTE:
# When creating points, the method starts at the first vertex of an edge,
# which is by convention the I-node for the digitized edge
# We need the IJ slope as well as the JI slope, so we will perform 2 iterations over edges to calculate upslope

In [44]:
# Loop through all edges
upslope_ij = {}
upslope_ji = {}
for edge in edges: # Replace [1] with edges when ready to loop over everything
    edge_df = df.query("PSRCEdgeID == " + str(edge))
    # Extract the elevation data to numPy because it's faster to loop over
    elev_data = edge_df['RASTERVALU'].values
    # Loop through each point in each edge
    upslope_ij[edge] = 0
    upslope_ji[edge] = 0
    for point in xrange(len(elev_data)-1):  # stop short of the list because we only want to compare the 2nd to last to last
        elev_diff = elev_data[point+1] - elev_data[point]
        if elev_diff > 0:
            upslope_ij[edge] += elev_diff
        elif elev_diff < 0:
            upslope_ji[edge] += abs(elev_diff)      # since we know it will be "negative" for the JI direction when calculated
                                                    # in references to the IJ direction

In [57]:
pd.DataFrame(data=[upslope_ij)

,1,2,3,5,7,8,20,26,27,29,...,159,160,161,163,164,165,169,170,171,172
0,371.592758,45.506660,49.597279,490.041498,35.044041,92.659582,12.459839,76.259247,687.290070,188.374573,...,25.803696,762.00293,6.127695,50.204651,1.074886,329.423828,0.000000,47.423370,4.887360,182.904175
1,262.753494,40.233576,27.654609,486.875498,76.479980,155.491697,11.084167,87.205307,546.604004,261.661469,...,7.101471,423.12500,9.827912,53.829132,1.803417,187.498413,3.199051,0.284809,12.959885,177.150879


In [71]:
# Import dictionary to a series and attach upslope back on the original dataframe
upslope_ij_s = pd.Series(upslope_ij, name='elev_gain_ij')
upslope_ji_s = pd.Series(upslope_ji, name='elev_gain_ji')
upslope_ij_s.index.name='edgeID'
upslope_ij_s = upslope_ij_s.reset_index()
upslope_ji_s.index.name='edgeID'
upslope_ji_s = upslope_ji_s.reset_index()

In [72]:
newdf = upslope_ij_s.merge(upslope_ji_s)

In [73]:
# add in edge length
newdf = newdf.merge(length)

In [74]:
# Calcualte the average upslope 
newdf['avg_upslope_ij'] = newdf['elev_gain_ij']/newdf['Shape_Length']
newdf['avg_upslope_ji'] = newdf['elev_gain_ji']/newdf['Shape_Length']

In [77]:
#newdf.to_csv('output_test1.csv')
newdf

,edgeID,elev_gain_ij,elev_gain_ji,Shape_Length,avg_upslope_ij,avg_upslope_ji
0,1,371.592758,262.753494,4886.087587,0.076051,0.053776
1,2,45.506660,40.233576,2464.140052,0.018468,0.016328
2,3,49.597279,27.654609,1196.499466,0.041452,0.023113
3,5,490.041498,486.875498,12235.034954,0.040052,0.039794
4,7,35.044041,76.479980,1783.305434,0.019651,0.042887
5,8,92.659582,155.491697,4516.027224,0.020518,0.034431
6,20,12.459839,11.084167,627.880691,0.019844,0.017653
7,26,76.259247,87.205307,880.944283,0.086565,0.098991
8,27,687.290070,546.604004,20492.551214,0.033539,0.026673
9,29,188.374573,261.661469,4693.105188,0.040139,0.055754


In [79]:
# Read in the final CSV
df = pd.read_csv('slope_output.csv')

In [80]:
len(df)

229828

In [81]:
df.columns

Index([u'Unnamed: 0', u'edgeID', u'elev_gain_ij', u'elev_gain_ji',
       u'Shape_Length', u'avg_upslope_ij', u'avg_upslope_ji'],
      dtype='object')

In [89]:
df['elev_gain_ij'].mean()

27.5354025801546

In [91]:
# Join the bike attributes data
bike_attr = r'D:\slope\test.gdb\tblBikeAttributes'

In [142]:
# Load in bike attributes
# Note that we can't have null fields- need to update the gdb to ensure this
bike_np = arcpy.da.TableToNumPyArray(bike_attr, ('PSRCEdgeID', 'IJBikeFacil', 'JIBikeFacil'))

In [133]:
# Laod numpy to pandas
bike_df = pd.DataFrame(bike_np)

In [208]:
bike_df

,PSRCEdgeID,IJBikeFacil,JIBikeFacil
0,242615,0,0
1,242754,0,0
2,242816,0,0
3,242993,0,0
4,243120,0,0
5,243348,0,0
6,243441,0,0
7,243653,0,0
8,244252,0,0
9,244262,0,0


In [378]:
# Load in the slope edge data
edge_df = pd.read_csv(r"D:\slope\slope_output.csv")

In [379]:
# Merge the bike data with the elevation data
total_merge = edge_df.merge(bike_df)

In [383]:
# Filter out the inactive links for now
# eventually we need to use a network that includes all of these links, but this is for testingy type 
total_merge = total_merge.query('ActiveLink == 1')

# Also need to add in facility type 17 (P&R connectors and ignore those for now)
total_merge = total_merge.query('FacilityTy <> 17')

total_merge = total_merge.query('Fullname <> "P&R walk link"')

In [384]:
# We need to break up the links into IJ and JI if the link is bi-directional

# IF direction is +1, use the IJ fields

oneway_ij_df = total_merge.query('Direction == 1')
oneway_ij_df.columns
# Set the bike facility attribute field equal to the IJ facility field
oneway_ij_df['@bkfac'] = oneway_ij_df['IJBikeFacil']

# Set the average upslope field equal to the IJ field
oneway_ij_df['@upslp'] = oneway_ij_df['avg_upslope_ij']
len(oneway_ij_df)

c:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1025

In [385]:
# If direction is -1, use the JI fields

oneway_ji_df = total_merge.query('Direction == -1')
oneway_ji_df.columns
# Set the bike facility attribute field equal to the IJ facility field
oneway_ji_df['@bkfac'] = oneway_ji_df['JIBikeFacil']

# Set the average upslope field equal to the IJ field
oneway_ji_df['@upslp'] = oneway_ji_df['avg_upslope_ji']
len(oneway_ji_df)

c:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2

In [386]:
# if direction is 0, use IJ field and create another row for the JI direction
# and assign IJ to the original field, JI to the copy
twoway_df_ij = total_merge.query('Direction == 0')

In [387]:
# For reverse direction, make a copy of the twoway_df, but flip the I and J node column names
twoway_df_ji['NewINode'] = twoway_df_ij['NewJNode']    # Copy j values from this table
twoway_df_ji['NewJNode'] = twoway_df_ij['NewINode']    # and copy i values from the j values in

In [388]:
# Add the attributes for each of these dataframes
twoway_df_ij['@bkfac'] = twoway_df_ij['IJBikeFacil']
twoway_df_ij['@upslp'] = twoway_df_ij['avg_upslope_ij']


twoway_df_ji['@bkfac'] = twoway_df_ji['JIBikeFacil']
twoway_df_ji['@upslp'] = twoway_df_ji['avg_upslope_ji']

c:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
c:\Users\Brice\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [389]:
# Combine all the dataframes
combined_df = pd.concat([twoway_df_ij, twoway_df_ji, oneway_ij_df, oneway_ji_df])

In [390]:
len(twoway_df_ij) + len(twoway_df_ji) + len(oneway_ij_df) + len(oneway_ji_df)

47635

In [391]:
# We need to do some clean up because there are some odd results due to edge creation
# For instance, the really short weave links have high slopes because they are such short segments
# These should be ignored and slope replaced with 0

# All the links with an edgeID of 98 should be removed from this set
combined_df_trimmed = combined_df[combined_df["edgeID"] <> 98]

In [392]:
len(combined_df)

47635

In [393]:
# Write results out to a space-delimited input for Emme
emme_attr = combined_df_trimmed[['NewINode', 'NewJNode', '@bkfac', '@upslp']]
emme_attr.columns = ['inode', 'jnode', '@bkfac', '@upslp']

In [394]:
# Remove nan rows
emme_attr = emme_attr[pd.notnull(emme_attr['inode'])]
emme_attr = emme_attr[pd.notnull(emme_attr['jnode'])]

In [395]:
# Node columns need to be integers
emme_attr['inode'] = emme_attr['inode'].astype('int')
emme_attr['jnode'] = emme_attr['jnode'].astype('int')

In [396]:
# Rename the Node columns

emme_attr.to_csv('emme_attr.in', sep=' ', index=False)

In [404]:
total_merge.query('IJBikeFacil == 2')

,Unnamed: 0,edgeID,elev_gain_ij,elev_gain_ji,Shape_Length_x,avg_upslope_ij,avg_upslope_ji,PSRCEdgeID,NewINode,NewJNode,ActiveLink,FacilityTy,Fullname,Direction,Shape_Length_y,IJBikeFacil,JIBikeFacil
6759,6759,44686,51.685486,0.000000,1383.880796,0.037348,0.000000,44686,86145,86906,1,6,244TH,0,1383.880796,2,2
7137,7137,48243,14.515994,12.693770,1754.458462,0.008274,0.007235,48243,91075,90464,1,6,ALKI,0,1754.458462,2,2
7167,7167,48536,1.082335,2.706211,707.972398,0.001529,0.003822,48536,91487,91309,1,6,ALKI,0,707.972398,2,2
7172,7172,48563,0.470341,2.292347,353.888365,0.001329,0.006478,48563,91309,91224,1,6,ALKI,0,353.888365,2,2
7173,7173,48564,0.985679,1.662665,383.532787,0.002570,0.004335,48564,91224,91075,1,6,ALKI,0,383.532787,2,2
16017,16017,200858,2.238853,0.543217,257.708942,0.008688,0.002108,200858,88056,87930,1,8,6TH,0,257.708942,2,2
18014,18014,204644,40.225244,40.571952,5610.263779,0.007170,0.007232,204644,87979,90464,1,6,ALKI,0,5610.263779,2,2


In [397]:
emme_attr['@upslp'].nlargest(5)

15811    22.375619
1113      2.825998
25465     2.270261
1076      2.224505
1076      2.177039
Name: @upslp, dtype: float64

In [377]:
emme_attr.loc[1381]

,inode,jnode,@bkfac,@upslp
1381,81733,81585,0,0.134287
1381,81585,81733,0,0.134604
